Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [ ]:
from collections import Counter
import numpy as np


In [ ]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"


## Basic Functions

In [ ]:
def sorted_valid_moves(board):
    center = NUM_COLUMNS // 2
    columns = [center]
    for i in range(center):
        i += 1
        col = center - i
        if col >= 0:
            columns.append(col)
        col = center + i
        if col < NUM_COLUMNS:
            columns.append(col)
    return [n for n in columns if board[n, COLUMN_HEIGHT - 1] == 0]


In [ ]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )


## Montecarlo Evaluation

In [ ]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)


## Minmax

In [ ]:
import math


### Heuristic static evaluation

Consider all the 4-in-a-row sequences without any opponent's piece and give them a score based on how many pieces they contain:
* 3 pieces -> 30 (3*10)
* 2 pieces -> 2
* 1 piece -> 1
* 0 pieces -> 0

The scores will carry the player's sign (+ or -)

In [ ]:
def eval_for_player(board, player):
    possibilites = np.array(
        [
            np.sum(board[c, r])
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
            if -player not in board[c, r]
        ]
        + [
            np.sum(board[c, r])
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
            if -player not in board[c, r]
        ]
        + [
            np.sum(board[diag])
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
            if -player not in board[diag]
        ]
        + [
            np.sum(board[diag])
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
            if -player not in board[diag]
        ]
    )

    return np.sum(np.where(np.abs(possibilites) == 3, possibilites * 10, possibilites))


def eval(board):
    return eval_for_player(board, 1) + eval_for_player(board, -1)


### Minmax with alpha-beta pruning

In [ ]:
def score_is_better(score, current_best, player):
    if player == 1:
        return score > current_best
    else:
        return score < current_best


def update_alpha_beta(alpha, beta, best_score, player):
    if player == 1:
        return max(alpha, best_score), beta
    else:
        return alpha, min(beta, best_score)


def minmax(board, depth, alpha, beta, player):
    if four_in_a_row(board, player):
        return None, (player * math.inf)
    elif four_in_a_row(board, -player):
        return None, (-player * math.inf)
    if depth == 0:
        return None, eval(board)
    best_score = -player * math.inf
    best_move = None
    for c in sorted_valid_moves(board):
        # if there are valid moves, initialize the best move to the first available
        # this way, if there are no good moves available, we will just choose the first one
        if best_move is None:
            best_move = c
        play(board, c, player)
        _, score = minmax(board, depth - 1, alpha, beta, -player)
        take_back(board, c)
        if score_is_better(score, best_score, player):
            best_score = score
            best_move = c
        alpha, beta = update_alpha_beta(alpha, beta, best_score, player)
        if alpha >= beta:
            break
    return best_move, best_score


## Example

In [ ]:
SYMBOLS = {1: "X", -1: "O", 0: "-"}


def print_board(board):
    for r in reversed(range(COLUMN_HEIGHT)):
        print("|", end=" ")
        for c in range(NUM_COLUMNS):
            print(SYMBOLS[board[c][r]], end=" ")
        print("|")


def check_win(board):
    for player in [1, -1]:
        if four_in_a_row(board, player):
            return player
    return 0


In [ ]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
player = 1
alpha = -math.inf
beta = math.inf

print_board(board)

winner = 0

while valid_moves(board):
    print()
    next_move, score = minmax(board, 5, alpha, beta, player)
    play(board, next_move, player)
    print_board(board)
    winner = check_win(board)
    if winner != 0:
        print(f"Player {SYMBOLS[winner]} won")
        break
    player = -player

if winner == 0:
    print("Draw")


## MCTS (NOT WORKING CORRECTLY)

In [ ]:
import random


In [ ]:
# Upper Confidence Bound
def ucb(s_p, s, w):
    return (w / s) + math.sqrt(math.log(s_p) / s)


In [ ]:
class node:
    def __init__(self, parent, board, player):
        self.state = board
        self.player = player
        self.children = set()
        self.parent = parent
        self.simulations = 0
        self.wins = 0

    def select(self):
        best_node = self
        while best_node.children and all(
            child.simulations > 0 for child in best_node.children
        ):
            best_node = max(best_node.children, key=lambda child: child.get_ucb())
        return best_node

    def expand(self):
        if not self.children:
            for c in valid_moves(self.state):
                play(self.state, c, self.player)
                self.children.add(node(self, np.copy(self.state), -self.player))
                take_back(self.state, c)

    def simulate(self):
        winner = _mc(np.copy(self.state), self.player)

        # backpropagation
        node = self
        while node is not None:
            node.simulations += 1
            if winner != node.player:
                node.wins += 1
            node = node.parent

    def get_ucb(self):
        if not self.parent:
            return None
        return ucb(self.parent.simulations, self.simulations, self.wins)


In [ ]:
def is_terminal(board):
    if not valid_moves(board):
        return True
    for p in [-1, 1]:
        if four_in_a_row(board, p):
            return True
    return False


def run_mcts(node):
    node = node.select()  # select node using ucb
    if is_terminal(node.state):
        return
    node.expand()
    node = random.choice([child for child in node.children if child.simulations == 0])
    node.simulate()


def get_best(node):
    if not node.children:
        return None
    return max(node.children, key=lambda child: child.wins / child.simulations)


In [ ]:
root = node(None, np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte), 1)
current_node = root

while valid_moves(current_node.state):
    print_board(current_node.state)
    for _ in range(100):
        run_mcts(current_node)
    for p in [-1, 1]:
        if four_in_a_row(current_node.state, p):
            print(f"Player {SYMBOLS[p]} won")
    current_node = get_best(current_node)
    if current_node is None:
        break
    print()
